In [ ]:
import requests
import pandas as pd
import io
from datetime import datetime, timedelta
import urllib.request
import gc

In [ ]:
## Pull Details of all Precision Monitoring Stations
r = requests.get('https://newcastle.urbanobservatory.ac.uk/api/v1.1/sensors/csv/?broker=aq_mesh_api&search=mon')
sensorInfo = pd.read_csv(io.StringIO(r.text) )
sensorList = sensorInfo["Sensor Name"].to_list()

In [ ]:
## Get current time, add 1 hour
now = datetime.now()
now = now + timedelta(hours=1)
now = now.strftime('%Y%m%d%H%M%S')

In [ ]:
combRecentData = pd.DataFrame()

In [ ]:
for s in sensorList:
    apiCall = 'https://newcastle.urbanobservatory.ac.uk/api/v1.1/sensors/'+s+'/data/csv/'
    ## Pull data from start of March onwards - approx start of emergency measures was 17th March   
    params = dict(starttime='20200301000000',endtime=now)

    r = requests.get(apiCall,params)
    if r.status_code != 404:
        sensorData = pd.read_csv(io.StringIO(r.text))
        combRecentData = combRecentData.append(sensorData, ignore_index=True)

In [ ]:
pd.to_pickle(combRecentData, '../cache/update-airquality-airmon-api.pkl')
gc.collect()